In [1]:
! pip install keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

In [2]:
def readData(dataFile,maxSequenceLength,embeddings):
	labels=[]
	sources=[]
	wordsD={'<PAD>':0,'<UNK>':1}
	wordsL=['<PAD>','<UNK>']
	contents=[]
	nums=[]
	documentCVs=[]
	topicCVs=[]
	sourceCVs=[]
	fileIn=open(dataFile,'r', encoding="UTF-8")
	while True:
		line=fileIn.readline()
		if line=='':
			break
		parts=line.strip().split('\t')
		label=parts[0]
		source=parts[1]
		num=parts[2]
		documentCV=parts[3]
		topicCV=parts[4]
		sourceCV=parts[5]
		contentWords=parts[6:]
		if len(contentWords)>maxSequenceLength:
			contentWords=contentWords[0:maxSequenceLength]
		content=[]
		for contentWord in contentWords:
			if not contentWord in embeddings:
				contentWord='<UNK>'
			elif not contentWord in wordsD:
				wordsD[contentWord]=len(wordsD)
				wordsL.append(contentWord)
			content.append(wordsD[contentWord])
		if len(content)<maxSequenceLength:
			content=content+[0]*(maxSequenceLength-len(content))	
		labels.append(label)
		sources.append(source)
		contents.append(content)
		nums.append(num)
		documentCVs.append(documentCV)
		topicCVs.append(topicCV)
		sourceCVs.append(sourceCV)
	fileIn.close()
	return(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)


def readDocuments(dataFile,maxSequenceLength,embeddings,maxDocumentLength):
	(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)=readData(dataFile,maxSequenceLength,embeddings)
	labelsD=[]
	sourcesD=[]
	numsD=[]
	contentsD=[]
	documentCVsD=[]
	topicCVsD=[]
	sourceCVsD=[]
	lengthsD=[]
	i=0
	contentHere=[]
	while(True):
		contentHere.append(contents[i])
		if i==len(nums)-1 or nums[i+1]!=nums[i]:
			labelsD.append(labels[i])
			sourcesD.append(sources[i])
			numsD.append(nums[i])
			documentCVsD.append(documentCVs[i])
			topicCVsD.append(topicCVs[i])
			sourceCVsD.append(sourceCVs[i])
			if len(contentHere)<maxDocumentLength:
				lengthHere=len(contentHere)
				for j in range(maxDocumentLength-len(contentHere)):
					contentHere.append([0]*maxSequenceLength)
			else:
				lengthHere=maxDocumentLength
				contentHere=contentHere[0:maxDocumentLength]
			lengthsD.append(lengthHere)
			contentsD.append(contentHere)
			contentHere=[]
		i=i+1
		if i==len(nums):
			break
	return(labelsD,sourcesD,numsD,wordsL,contentsD,documentCVsD,topicCVsD,sourceCVsD,lengthsD)


def readEmbeddings(vectorsFile):
	embeddings={}
	for line in open(vectorsFile, encoding='UTF-8'):
		parts=line.split()
		if len(parts)!=301:
			continue
		coefs=np.asarray(parts[1:],dtype='float32')
		embeddings[parts[0]]=coefs
	return(embeddings)

def prepareEmbeddings(embeddings,wordsL):
	embeddingMatrix = np.zeros((len(wordsL), 300))
	for i in range(len(wordsL)):
		row=np.zeros(300)
		word=wordsL[i]
		if word!='<PAD>' and word!='<UNK>':
			row=embeddings[word]
			row=row/np.sqrt(sum(row*row))
		embeddingMatrix[i]=row
	return(embeddingMatrix)

def lengthToAverageMask(lengthD,maxDocumentLength,binary=False):
	result=[]
	for i in range(len(lengthD)):
		if binary:
			multiplier=1.0
		else:
			multiplier=maxDocumentLength*1.0/lengthD[i]
		vec=[multiplier,multiplier]
		vec0=[0.0,0.0]
		vector=[vec]*lengthD[i]+[vec0]*(maxDocumentLength-lengthD[i])
		result.append(vector)
	return(np.array(result))

	
def evaluateD(resultPred,resultTrue):
	return(np.mean((resultPred[:,1]>0.5)==(resultTrue[:,1]==1)))






class Style1():
	def __init__(self,embeddingMatrix,labelsNum,maxSequenceLength,maxDocumentLength,onGPU):
		self.maxSequenceLength=maxSequenceLength
		self.maxDocumentLength=maxDocumentLength
		self.labelsNum=labelsNum
		
		self.embeddingSize=np.shape(embeddingMatrix)[1]
		self.embL=keras.layers.Embedding(np.shape(embeddingMatrix)[0],self.embeddingSize,input_length=maxSequenceLength,weights=[embeddingMatrix],trainable=False)
		self.reshape1L=keras.layers.Lambda(self.backend_reshape,output_shape=(maxSequenceLength,self.embeddingSize))
		if onGPU:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		else:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		self.conL=keras.layers.Concatenate(axis=1)
		self.denseL=keras.layers.Dense(labelsNum,activation="softmax")
		self.reshape2L=keras.layers.Lambda(self.backend_reshape2,output_shape=(maxDocumentLength,labelsNum))
		self.multiply=keras.layers.Multiply()
		self.poolingL=keras.layers.GlobalAveragePooling1D()

	def backend_reshape(self,x):
		return keras.backend.reshape(x,(-1,self.maxSequenceLength,self.embeddingSize))

	def backend_reshape2(self,x):
		return keras.backend.reshape(x,(-1,self.maxDocumentLength,self.labelsNum))

	def getMask(self,lengthD):
		return (lengthToAverageMask(lengthD,self.maxDocumentLength,binary=False))

	def getModel(self):
		inputWords = keras.layers.Input(shape=(self.maxDocumentLength,self.maxSequenceLength,))
		inputMask = keras.layers.Input(shape=(self.maxDocumentLength,self.labelsNum,))
		x=self.embL(inputWords)
		x=self.reshape1L(x)
		lstm1=self.LSTMforL(x)
		lstm2=self.LSTMrevL(x)
		x=self.conL([lstm1,lstm2])
		Ps=self.denseL(x)
		Ps=self.reshape2L(Ps)
		Ps=self.multiply([Ps,inputMask])
		Ps=self.poolingL(Ps)
		model=Model(inputs=[inputWords,inputMask], outputs=Ps)
		return(model)




In [3]:
# Local setup
dataPath="D:\\BiLSTMAvg\\final_file\\"
batch_size=64
onGPU=True	
epochs=5
dataset="all_data.tsv"

print('Reading data')
# Reading data

embeddings=readEmbeddings(dataPath+"GoogleNewsUnigrams.txt")
MAX_SEQUENCE_LENGTH=120
MAX_DOCUMENT_LENGTH=50
(labels,sources,nums,wordsL,contents,documentCV,topicCV,sourceCV,lengthD)=readDocuments(dataPath+dataset,MAX_SEQUENCE_LENGTH,embeddings,MAX_DOCUMENT_LENGTH)
embeddingMatrix=prepareEmbeddings(embeddings,wordsL)

print("Converting to numpy")
# Converting to numpy
y=np.asarray(labels,dtype='float32')
allY=np.concatenate((np.expand_dims(1-y,1),np.expand_dims(y,1)),axis=1)
allX=np.array(contents)

print('Loading models')
# Load models 

Reading data
Converting to numpy
Loading models


In [4]:
# Prepare CV scenarios
documentCV=np.asarray(documentCV,dtype='int32')
topicCV=np.asarray(topicCV,dtype='int32')
sourceCV=np.asarray(sourceCV,dtype='int32')


In [5]:
scenarioCV=topicCV

print('Start')
result=np.array([[-1,-1]]*len(scenarioCV),dtype='float32')
for folda in range(max(scenarioCV)):
	fold=folda+1
	print("Evaluating on fold "+str(fold)+"...")
	whichTest=np.isin(scenarioCV,fold)
	trainY=allY[~whichTest,]
	develY=allY[whichTest,]
	trainX=allX[~whichTest,]
	develX=allX[whichTest,]
	style1=Style1(embeddingMatrix,2,MAX_SEQUENCE_LENGTH,MAX_DOCUMENT_LENGTH,onGPU)
	mask=style1.getMask(lengthD)
	trainM=np.array(mask)[~whichTest]
	develM=np.array(mask)[whichTest]
	model=style1.getModel()
	model.compile(optimizer=tf.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])
	fit=model.fit([trainX,trainM],trainY, epochs=epochs,validation_data=([develX,develM],develY),batch_size=batch_size)
	predictions=model.predict([develX,develM])
	result[whichTest,:]=predictions

evaluateD(result,allY)

Start
Evaluating on fold 1...
Epoch 1/5
779/779 [==============================] - 8000s 10s/step - loss: 0.4735 - accuracy: 0.7794 - val_loss: 0.3929 - val_accuracy: 0.8250
Epoch 2/5
779/779 [==============================] - 7913s 10s/step - loss: 0.4013 - accuracy: 0.8268 - val_loss: 0.3817 - val_accuracy: 0.8206
Epoch 3/5
779/779 [==============================] - 7881s 10s/step - loss: 0.3855 - accuracy: 0.8354 - val_loss: 0.3722 - val_accuracy: 0.8526
Epoch 4/5
779/779 [==============================] - 7748s 10s/step - loss: 0.3724 - accuracy: 0.8460 - val_loss: 0.3610 - val_accuracy: 0.8619
Epoch 5/5
236/236 [==============================] - 499s 2s/step
Evaluating on fold 2...
Epoch 1/5


: 

: 

In [6]:
scenarioCV=documentCV

print('Start')
result=np.array([[-1,-1]]*len(scenarioCV),dtype='float32')
for folda in range(max(scenarioCV)):
	fold=folda+1
	print("Evaluating on fold "+str(fold)+"...")
	whichTest=np.isin(scenarioCV,fold)
	trainY=allY[~whichTest,]
	develY=allY[whichTest,]
	trainX=allX[~whichTest,]
	develX=allX[whichTest,]
	style1=Style1(embeddingMatrix,2,MAX_SEQUENCE_LENGTH,MAX_DOCUMENT_LENGTH,onGPU)
	mask=style1.getMask(lengthD)
	trainM=np.array(mask)[~whichTest]
	develM=np.array(mask)[whichTest]
	model=style1.getModel()
	model.compile(optimizer=tf.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])
	fit=model.fit([trainX,trainM],trainY, epochs=epochs,validation_data=([develX,develM],develY),batch_size=batch_size)
	predictions=model.predict([develX,develM])
	result[whichTest,:]=predictions

evaluateD(result,allY)

Start
Evaluating on fold 1...
Epoch 1/5
717/717 [==============================] - 7741s 11s/step - loss: 0.4749 - accuracy: 0.7770 - val_loss: 0.4124 - val_accuracy: 0.8040
Epoch 2/5
717/717 [==============================] - 7547s 11s/step - loss: 0.4042 - accuracy: 0.8239 - val_loss: 0.3908 - val_accuracy: 0.8373
Epoch 3/5
717/717 [==============================] - 7730s 11s/step - loss: 0.3873 - accuracy: 0.8330 - val_loss: 0.4351 - val_accuracy: 0.8190
Epoch 4/5
717/717 [==============================] - 7708s 11s/step - loss: 0.3744 - accuracy: 0.8434 - val_loss: 0.3696 - val_accuracy: 0.8596
Epoch 5/5
361/361 [==============================] - 742s 2s/step
Evaluating on fold 2...
Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'while/MatMul' defined at (most recent call last):
    File "d:\Anaconda\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\Anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "d:\Anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "d:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "d:\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "d:\Anaconda\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "d:\Anaconda\lib\asyncio\base_events.py", line 1890, in _run_once
      handle._run()
    Fi